In [1]:
import urllib.request
import json
import os
import ssl
import base64
import credential

In [2]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

In [3]:
source = "C:/Users/rabiswas/Documents/Personal/Bellevue/DSC630_PredictiveAnalytics/CourseProject/castingdata/casting_dataset/ok_front/cast_ok_0_7588.jpeg"
#base 64 encode and then decode as string
with open(source, "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode()

In [4]:
post_image = 'data:image/jpeg;base64,'+encoded_string

In [5]:
# Request data goes here
data = {
    "Inputs": {
        "WebServiceInput0":
        [
            {
                'image' : post_image,
                'id': "",
                'category': "",
            },
        ],
    },
    "GlobalParameters": {
    }
}

body = str.encode(json.dumps(data))

In [6]:
url = 'http://20.72.112.3:80/api/v1/service/casting-classification-endpoint/score'
api_key = credential.login['api_key']   # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
req = urllib.request.Request(url, body, headers)

In [7]:
try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'{"Results": {"WebServiceOutput0": [{"category": "", "id": "", "Scored Probabilities_def_front": 1.110250195779372e-05, "Scored Probabilities_ok_front": 0.9999889135360718, "Scored Labels": "ok_front"}]}}'


In [8]:
resp_json = json.loads(result)
resp_json

{'Results': {'WebServiceOutput0': [{'category': '',
    'id': '',
    'Scored Probabilities_def_front': 1.110250195779372e-05,
    'Scored Probabilities_ok_front': 0.9999889135360718,
    'Scored Labels': 'ok_front'}]}}

In [9]:
def parseClassificationResults(resp_json):
    predResults = resp_json["Results"]
    #print(predResults)
    if predResults['WebServiceOutput0'][0]["Scored Probabilities_def_front"] <= predResults['WebServiceOutput0'][0]["Scored Probabilities_ok_front"]:
        return "OK CAST"
    else:
        return "DEFECTIVE CAST"
    return None

In [10]:
parseClassificationResults(resp_json)

'OK CAST'